In [21]:
import pandas as pd
file_path='X:\hemlo\car_price_prediction.csv'
data=pd.read_csv(file_path)
columns_to_encode = ['Gear box type', 'Fuel type', 'Color', 'Manufacturer', 'Category', 'Leather interior', 'Drive wheels','Doors','Wheel']
data_encoded = pd.get_dummies(data, columns=columns_to_encode)


             ID  Price  Levy    Model  Prod. year Engine volume    Mileage  \
0      45654403  13328  1399   RX 450        2010           3.5  186005 km   
1      44731507  16621  1018  Equinox        2011             3  192000 km   
2      45774419   8467     -      FIT        2006           1.3  200000 km   
3      45769185   3607   862   Escape        2011           2.5  168966 km   
4      45809263  11726   446      FIT        2014           1.3   91901 km   
...         ...    ...   ...      ...         ...           ...        ...   
19232  45798355   8467     -  CLK 200        1999     2.0 Turbo  300000 km   
19233  45778856  15681   831   Sonata        2011           2.4  161600 km   
19234  45804997  26108   836   Tucson        2010             2  116365 km   
19235  45793526   5331  1288  Captiva        2007             2   51258 km   
19236  45813273    470   753   Sonata        2012           2.4  186923 km   

       Cylinders  Airbags  Gear box type_Automatic  ...  Leathe